## Get 5 road segments that are traversed the most often

In [1]:
import pandas as pd
# import shapely.wkt

mr = pd.read_csv('data/mr.csv', sep=';', index_col=0)
mr.sort_index(inplace=True)
mr = mr.loc[mr['mgeom'] != 'LINESTRING()']
# mr['mgeom'] = mr['mgeom'].apply(shapely.wkt.loads)
mr

,opath,cpath,mgeom
id,,,
1,"1095,1095,2732,2732,1093,1093,698,698,696,8687...","1095,2732,1093,698,8704,62550,8703,696,8687,17...","LINESTRING(-8.61862328377 41.1414555366,-8.618..."
2,"56205,56205,4790,70940,762,26201,26213,26202,5...","56205,69910,69917,69912,4790,56207,26241,70940...","LINESTRING(-8.63985895545 41.159750087,-8.6405..."
5,"67262,67262,67260,5878,5880,61719,765,70690,73...","67262,67260,57887,67265,5875,5878,5880,41236,6...","LINESTRING(-8.6457599 41.1805278,-8.645788 41...."
6,"62561,58603,701,26471,55235,55235,55235,55235,...","62561,58603,58083,65094,57135,701,26471,55235,...","LINESTRING(-8.61551679314 41.140715625,-8.6154..."
7,"993,1015,1015,3529,5139,69303,69300,40862,4086...","993,1015,3529,10723,5139,10772,69304,69303,693...","LINESTRING(-8.579496154 41.1459330735,-8.57952..."
...,...,...,...
994,"56938,56938,56938,56938,56938,56938,56936,5693...","56938,56936,1645,64636,1519,55259,55256,1389,1...","LINESTRING(-8.59697443774 41.1507050705,-8.596..."
995,"56205,56205,56205,56205,69908,10482,4788,68423...","56205,69911,69908,69915,56208,10482,68418,4788...","LINESTRING(-8.63982479396 41.1597447069,-8.640..."
998,"69300,69300,40862,40862,68548,40863,69301,6930...","69300,40862,68548,68549,40865,40863,40862,6854...","LINESTRING(-8.58610277406 41.1485589137,-8.586..."


In [2]:
road_segments = mr['cpath'].tolist()
road_segments = ",".join(road_segments).split(",")
road_segments = [int(i) for i in road_segments]
road_segments.sort()

In [6]:
import collections
counter = collections.Counter(road_segments)
result = counter.most_common(5)
result = [i[0] for i in result]
print("5 FIDs of the road segments which are traversed most often:")
print(result)

5 FIDs of the road segments which are traversed most often:
[2481, 608, 2433, 2434, 2489]


## Visualize these road segments

In [17]:
import osmnx as ox
import shapely.wkt

G = ox.load_graphml('data/porto-bbox.graphml')

edges = pd.read_csv('data/porto-bbox/edges.csv', index_col='fid')['WKT']
edges = edges[edges.index.isin(result)]
edges = edges.apply(shapely.wkt.loads)
edges


fid
608     LINESTRING (-8.6678023 41.2363728, -8.66791560...
2433    LINESTRING (-8.664968999999999 41.2261891, -8....
2434    LINESTRING (-8.666010200000001 41.2271823, -8....
2481    LINESTRING (-8.6680157 41.2299263, -8.6679482 ...
2489    LINESTRING (-8.6628618 41.2239767, -8.6639233 ...
Name: WKT, dtype: object

In [18]:
import numpy as np
coords = [np.array([list(i) for i in edges.iloc[j].coords]) for j in range(len(edges.index))]


In [15]:
import matplotlib.pyplot as plt

# Draw the basemap
fig, ax = ox.plot_graph(G, node_size=0, show=False, close=False, figsize=(48, 48))

# Get colors for each of the first 10 trajectories
color_list = ox.plot.get_colors(5, cmap='rainbow', return_hex=True)

# Plot the raw GPS points on the basemap
for idx, c in enumerate(color_list):
    plt.plot(*coords[idx].T, color=c, linewidth=4)

# Save the map as an image file
fig.savefig('output/task5-1.png', bbox_inches='tight', pad_inches=0)
# plt.show()
plt.close(fig)